# Programação Orientada a Objetos (POO)
## Tema 7 – Bases de Dados Relacionais

### Parte III – Acesso a BD SQLite com Python

Jaime A. Martins

(CEOT/ISE/UAlg - jamartins@ualg.pt)

###### Autores: Jaime Martins [v2]; Pedro Cardoso [v1]

## Estabelecimento de conexão à base de dados usando um Connector/Python

Criar a conexão usando o método `connect()` do módulo `sqlite3`, que tem como parâmetro o caminho para o ficheiro que contém a base de dados 

In [1]:
import sqlite3

# Se ainda não existir a base de dados (ficheiro), será criado
conn = sqlite3.connect("exemplo.db")

conn.close()

## Criação de uma base de dados

Para a criação das tabelas e relacionamentos podemos construir o SQL statement ou, como alternativa, podemos usar ferramentas como sejam o MySQL Workbench, DbSchema, o Phpmyadmin, o SQlite Browser, o DataGrip, etc. 


Consideremos o caso em que contruímos o SQL statement...

In [2]:
sql = """
CREATE TABLE
    Location (
        idLocation INTEGER CONSTRAINT Location_pk PRIMARY key autoincrement,
        name TEXT NOT NULL,
        description text NOT NULL
    );

CREATE TABLE
    Unit (
        unit text CONSTRAINT Unit_pk PRIMARY key,
        description text NOT NULL
    );

CREATE TABLE
    Sensor (
        idSensor INTEGER CONSTRAINT Sensor_pk PRIMARY key autoincrement,
        idLocation INTEGER NOT NULL CONSTRAINT Sensor_Location_idLocation_fk REFERENCES Location ON UPDATE cascade ON DELETE cascade,
        name text NOT NULL,
        unit text NOT NULL CONSTRAINT Sensor_Unit_unit_fk REFERENCES Unit ON UPDATE cascade ON DELETE cascade
    );

CREATE TABLE
    Reading (
        idReading INTEGER CONSTRAINT Reading_pk PRIMARY key autoincrement,
        idSensor INTEGER CONSTRAINT Reading_Sensor_idSensor_fk REFERENCES Sensor ON UPDATE cascade ON DELETE cascade,
        TIMESTAMP datetime DEFAULT CURRENT_TIMESTAMP,
        VALUE REAL NOT NULL
    );

CREATE TABLE
    Alert (
        idAlert INTEGER CONSTRAINT Alert_pk PRIMARY key autoincrement,
        idSensor INTEGER CONSTRAINT Alert_Sensor_idSensor_fk REFERENCES Sensor ON UPDATE cascade ON DELETE cascade,
        description text NOT NULL,
        cleared INTEGER
    )
"""

In [3]:
with sqlite3.connect("sensors.db") as conn:
    cursor = conn.cursor()
    
    # executescript is a nonstandard convenience method for executing multiple SQL statements at once.
    cursor.executescript(sql)
    
    #cursor.close() # not needed, as it is closed automatically when the with block ends

## Operações CRUD

### `INSERT` 

Aberta a conexão em sqlite

In [4]:
conn = sqlite3.connect("sensors.db")
cursor = conn.cursor()

inserir uma nova localização na base de dados e obter o id correspondente

In [5]:
# Prepare the sql statement for the new location
sql = """
    INSERT INTO location (name, description) 
    VALUES (?, ?)
"""

data = ("Prometheus Server", "Prometheus Server @ lab. 163 / ISE /UAlg")

# Execute the sql statement and get the new location id
cursor.execute(sql, data)

location_id = cursor.lastrowid
location_id

1

Quando estamos a usar um sistema transacional, temos de efetuar o `commit` depois de fazer um `INSERT`, `DELETE`, ou `UPDATE`.

Note-se que:
* O `commit` confirma a transação atual. Se não se chamar, tudo o que se fez desde a última chamada do `commit()` não será visível às outras conexões.
* Quando a BD é acedida por várias conexões e um dos processos modifica-a, a BD SQLite fica bloqueada até que a transação seja confirmada (_commited_).
* Podemos desfazer as alterações desde o último `commit` chamando o método `rollback()`

In [6]:
conn.commit()

Inserir uma nova `Unit`

In [7]:
sql = """
    INSERT INTO Unit (unit, description) 
    VALUES (?, ?)
"""

data = ("percent", "percentage of usage")

cursor.execute(sql, data)  # statement + tupla com os dados
conn.commit()

Inserir um novo sensor e obter o seu id:
   1. Preparar o sql, note-se os _placeholders_ com `:nome` usados neste caso (`?` vs `:nome`)
   1. Preparar os dados 
   1. Executar a instrução `execute()` com os valores dos _placeholders_

In [8]:
sql = """
    INSERT INTO Sensor (idLocation, name, unit)
    VALUES (:idLocation, :name, :unit)
"""

data = {
    "idLocation": location_id,
    "name": "cpu_sensor_01",
    "unit": "percent",
}

cursor.execute(sql, data)
sensor_id = cursor.lastrowid
conn.commit()
sensor_id

1

E agora, obter alguns dados e enviar para a base de dados

In [9]:
import psutil

sql = """
    INSERT INTO Reading (idSensor, value)     
    VALUES (:idSensor, :value)
"""

for _ in range(20):
    data = {
        "idSensor": sensor_id,
        "value": psutil.cpu_percent(interval=1),
    }
    cursor.execute(sql, data)
    conn.commit()
    print(".", end="")

....................

In [10]:
cursor.close()
conn.close()

## Selecionar dados

In [11]:
import sqlite3

conn = sqlite3.connect("sensors.db")
cursor = conn.cursor()

In [12]:
sql = """
    SELECT idLocation, name, description 
    FROM location 
    WHERE description LIKE "%163%"
"""

cursor.execute(sql)

for (idLocation, name, description) in cursor:
    print(f"id: {idLocation}\n\t name: {name} \n\t description: {description}")

id: 1
	 name: Prometheus Server 
	 description: Prometheus Server @ lab. 163 / ISE /UAlg


In [13]:
sql = """
    SELECT idReading, idSensor, timestamp, value 
    FROM reading 
    WHERE value BETWEEN ? and ?
"""
data = (5, 30)

cursor.execute(sql, data)

for (idReading, idSensor, timestamp, value) in cursor:
    print(f"idReading: {idReading}\n\t idSensor: {idSensor} \n\t time: {timestamp} \n\t value: {value}")

idReading: 5
	 idSensor: 1 
	 time: 2023-03-17 15:53:10 
	 value: 7.4
idReading: 18
	 idSensor: 1 
	 time: 2023-03-17 15:53:25 
	 value: 5.8


In [14]:
sql = """
    SELECT *
    FROM Location as L
    INNER JOIN Sensor AS S ON L.idLocation = S.idLocation
    INNER JOIN Unit AS U ON S.unit = U.unit
    INNER JOIN Reading AS R ON S.idSensor = R.idSensor
    WHERE VALUE BETWEEN :low AND :high
    ORDER BY VALUE
"""

data = {
    "low": 1,
    "high": 2,
}

cursor.execute(sql, data)

Podemos obter os nomes das colunas

In [15]:
cursor.description

(('idLocation', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('description', None, None, None, None, None, None),
 ('idSensor', None, None, None, None, None, None),
 ('idLocation', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('unit', None, None, None, None, None, None),
 ('unit', None, None, None, None, None, None),
 ('description', None, None, None, None, None, None),
 ('idReading', None, None, None, None, None, None),
 ('idSensor', None, None, None, None, None, None),
 ('TIMESTAMP', None, None, None, None, None, None),
 ('VALUE', None, None, None, None, None, None))

In [16]:
lista_de_colunas = [linha[0] for linha in cursor.description]
lista_de_colunas

['idLocation',
 'name',
 'description',
 'idSensor',
 'idLocation',
 'name',
 'unit',
 'unit',
 'description',
 'idReading',
 'idSensor',
 'TIMESTAMP',
 'VALUE']

In [17]:
for linha in cursor:
    print(
        "\t".join(
            [f"|{coluna}: {valor}" for valor, coluna in zip(linha, lista_de_colunas)]
        )
    )

|idLocation: 1	|name: Prometheus Server	|description: Prometheus Server @ lab. 163 / ISE /UAlg	|idSensor: 1	|idLocation: 1	|name: cpu_sensor_01	|unit: percent	|unit: percent	|description: percentage of usage	|idReading: 18	|idSensor: 1	|TIMESTAMP: 2023-03-17 15:53:25	|VALUE: 5.8
|idLocation: 1	|name: Prometheus Server	|description: Prometheus Server @ lab. 163 / ISE /UAlg	|idSensor: 1	|idLocation: 1	|name: cpu_sensor_01	|unit: percent	|unit: percent	|description: percentage of usage	|idReading: 5	|idSensor: 1	|TIMESTAMP: 2023-03-17 15:53:10	|VALUE: 7.4


Usando o comando `fetchall` podemos obter todos os resultados de uma única vez como uma lista de tuplas

In [18]:
# É necessário voltar a correr o SELECT pois o cursor foi esvaziado
cursor.execute(sql, data)

cursor.fetchall()

[(1,
  'Prometheus Server',
  'Prometheus Server @ lab. 163 / ISE /UAlg',
  1,
  1,
  'cpu_sensor_01',
  'percent',
  'percent',
  'percentage of usage',
  18,
  1,
  '2023-03-17 15:53:25',
  5.8),
 (1,
  'Prometheus Server',
  'Prometheus Server @ lab. 163 / ISE /UAlg',
  1,
  1,
  'cpu_sensor_01',
  'percent',
  'percent',
  'percentage of usage',
  5,
  1,
  '2023-03-17 15:53:10',
  7.4)]

Podemos também converter para um dicionário mas __nosso caso NÃO é boa ideia__ pois duas colunas "têm o mesmo nome" (e.g., `nome`), pelo que se perdem colunas.

In [19]:
# É necessário voltar a correr o SELECT pois o cursor foi esvaziado
cursor.execute(sql, data)

for linha in cursor:
    print({coluna: valor for valor, coluna in zip(linha, lista_de_colunas)})

{'idLocation': 1, 'name': 'cpu_sensor_01', 'description': 'percentage of usage', 'idSensor': 1, 'unit': 'percent', 'idReading': 18, 'TIMESTAMP': '2023-03-17 15:53:25', 'VALUE': 5.8}
{'idLocation': 1, 'name': 'cpu_sensor_01', 'description': 'percentage of usage', 'idSensor': 1, 'unit': 'percent', 'idReading': 5, 'TIMESTAMP': '2023-03-17 15:53:10', 'VALUE': 7.4}


In [20]:
cursor.close()
conn.close()

### EXERCÍCIO
Utilize os pacotes `time` e `psutil` para calcular a memoria virtual livre a cada segundo durante 30 segundos, guardando na base de dados. Use
```
...
x = psutil.virtual_memory()
x.free
...
```